In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Adrenocortical_Cancer"
cohort = "GSE67766"

# Input paths
in_trait_dir = "../../input/GEO/Adrenocortical_Cancer"
in_cohort_dir = "../../input/GEO/Adrenocortical_Cancer/GSE67766"

# Output paths
out_data_file = "../../output/preprocess/Adrenocortical_Cancer/GSE67766.csv"
out_gene_data_file = "../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE67766.csv"
out_clinical_data_file = "../../output/preprocess/Adrenocortical_Cancer/clinical_data/GSE67766.csv"
json_path = "../../output/preprocess/Adrenocortical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Cancer Cells Hijack PRC2 to Modify Multiple Cytokine Pathways"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell line: SW-13']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title and summary, we cannot definitively determine if gene expression data is available
# The sample characteristics only mention "cell line: SW-13" which doesn't tell us about the type of data
# Since there's no clear indication that this is gene expression data (vs miRNA or methylation)
# and the series is described as a "SuperSeries composed of the SubSeries", 
# we should err on the cautious side
is_gene_available = False

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# There's no trait data specific to Adrenocortical_Cancer in the sample characteristics
trait_row = None

# Age is not available in the sample characteristics
age_row = None

# Gender is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# Define conversion functions for completeness, though they won't be used in this case
def convert_trait(value):
    # This won't be used as trait_row is None
    return None

def convert_age(value):
    # This won't be used as age_row is None
    return None

def convert_gender(value):
    # This won't be used as gender_row is None
    return None

# 3. Save Metadata
# Conduct initial filtering and save the metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# The identifiers starting with "ILMN_" are Illumina BeadArray probe IDs, not human gene symbols
# These are proprietary identifiers used by Illumina microarray platforms
# They need to be mapped to standard human gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1825594', 'ILMN_1810803', 'ILMN_1722532', 'ILMN_1884413', 'ILMN_1906034'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['Unigene', 'RefSeq', 'RefSeq', 'Unigene', 'Unigene'], 'Search_Key': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'Transcript': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'ILMN_Gene': ['HS.388528', 'LOC441782', 'JMJD1A', 'HS.580150', 'HS.540210'], 'Source_Reference_ID': ['Hs.388528', 'XM_497527.2', 'NM_018433.3', 'Hs.580150', 'Hs.540210'], 'RefSeq_ID': [nan, 'XM_497527.2', 'NM_018433.3', nan, nan], 'Unigene_ID': ['Hs.388528', nan, nan, 'Hs.580150', 'Hs.540210'], 'Entrez_Gene_ID': [nan, '441782', '55818', nan, nan], 'GI': ['23525203', '89042416', '46358420', '7376124', '5437312'], 'Accession': ['BU678343', 'XM_497527.2', 'NM_018433.3', 'AW629334', 'AI818233'], 'Symbol': [nan, 'LOC441782', 'JMJD1A', nan, nan], 'Prot

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identifying the key columns for gene mapping
# From the annotation preview, we can see that:
# - 'ID' column contains ILMN identifiers that match our gene expression data
# - 'Symbol' column contains the gene symbols we want to map to

# 2. Create the gene mapping dataframe
prob_col = 'ID'
gene_col = 'Symbol'
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Print a preview of the mapping
print("Gene mapping preview (first 5 rows):")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print the first few gene symbols and their data
print("\nGene expression data after mapping (first 5 genes):")
print(gene_data.head())

# Print the shape of the gene expression data
print(f"\nShape of gene expression data: {gene_data.shape}")


Gene mapping preview (first 5 rows):
             ID       Gene
1  ILMN_1810803  LOC441782
2  ILMN_1722532     JMJD1A
6  ILMN_1708805      NCOA3
8  ILMN_1672526  LOC389834
9  ILMN_2185604   C17orf77

Gene expression data after mapping (first 5 genes):
         GSM1652385  GSM1652386  GSM1652387  GSM1652388  GSM1652389  \
Gene                                                                  
A1BG      229.98830   248.54450   237.17420   248.39860   249.26820   
A2BP1     318.77866   332.46035   345.21173   338.50507   351.49917   
A2M        70.80764    70.58915    63.49310    75.62191    72.35401   
A2ML1      73.61642    70.96689    79.69693    71.65809    74.31523   
A3GALT2   206.22774   193.16380   218.59780   188.89192   202.47758   

         GSM1652390  GSM1652391  GSM1652392  GSM1652393  GSM1652394  ...  \
Gene                                                                 ...   
A1BG      244.07600   258.36630   263.38710   258.61440   258.33540  ...   
A2BP1     345.69635   

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols using NCBI Gene database...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization, gene data shape: {gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Since we already determined that trait data is not available (is_trait_available=False in step 2),
# we should not attempt to link clinical and genetic data or process them further
print("Clinical data not available, dataset marked as unusable for trait-gene association studies.")

# Since we cannot perform final validation without clinical data, we need to use is_final=False
# We're recording information about gene data availability but not performing full validation
is_usable = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

print("Dataset is not usable for trait-gene association studies.")

Normalizing gene symbols using NCBI Gene database...
After normalization, gene data shape: (17551, 24)


Normalized gene expression data saved to ../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE67766.csv
Clinical data not available, dataset marked as unusable for trait-gene association studies.
Dataset is not usable for trait-gene association studies.
